<h1> 1.Časť - prieskumná analýza </h1>

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats


In [ ]:
dataP = pd.read_csv("personal_train.csv", index_col=0)
dataP.info()

In [ ]:
dataO = pd.read_csv("other_train.csv", index_col=0, na_values =["??"])
dataO.info()

In [ ]:
dataO.shape[0] - dataO.dropna().shape[0]

In [ ]:
dataP[dataP.isnull().any(axis=1)]

In [ ]:
dataP.head() 

In [ ]:
dataO.head()

In [ ]:
dataP.describe()

In [ ]:
dataO.describe()

In [ ]:
dataO.describe(exclude=np.number)

In [ ]:
dataP.median()

In [ ]:
dataO.median()
#dat prec kategorické

In [ ]:
dataP.plot.box()

In [ ]:
dataO.plot.box()

In [ ]:
dataP.age.plot.hist(bins=30)

In [ ]:
dataP2 = dataP[(dataP['age']>0) & (dataP['age'] < 100)]

In [ ]:
dataP2.age.plot.hist(bins=30)

In [ ]:
#co vsetko histogramovat z tychto dat? treba to nejako upravovat??
dataO.std_oxygen.plot.hist(bins=30)

In [ ]:
#STATISTIKA - KATEGORICKE ATRIBUTY
#doplnanie dat? vysoke veky ale je datum narodenia doplnit z toho???????????
# pokial chceme spravit napr class - ma to problem s tým ze je to cislo, ako to obísť??
dataP.sex.value_counts().plot(kind = 'pie')

In [ ]:
sns.pairplot(dataO, hue = 'occupation')

In [ ]:
dataO.corr()